In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.backend import sigmoid
from sklearn.metrics import confusion_matrix
from keras.wrappers.scikit_learn import KerasClassifier
import matplotlib.pyplot as plt
from math import sqrt
import threading
from multiprocessing.pool import ThreadPool
import time
import multiprocessing
import traceback

import prettytable

In [ ]:
import warnings; warnings.simplefilter('ignore')

In [ ]:
def pre_process_data(data,null_threshold):
    """
    Drops Date and Unix Date columns from the data.
    Drops the columns which has null values more than specified null_threshold.
    Replaces infinite values with NAN.
    Drops the rows which has null values.

    Parameters
    ----------
    data : dataframe

    null_threshold : numeric
        numeric value describing the amount of null values that can be present.

    Returns
    -------
    data : dataframe
        an updated dataframe after performing all the opertaions.
    """
    
    data.drop(columns=['Unix Date','Date'],axis=1,inplace=True)
    total = data.shape[0]
    for col in data.columns:
        if null_threshold * total / 100 < data[col].isnull().sum():
            data.drop(columns=[col],axis=1,inplace=True)
    data.replace([np.inf, -np.inf], np.nan, inplace=True)
    data = data.apply(pd.to_numeric,errors='coerce')
    data.dropna(axis=0,inplace=True)
    return data

In [ ]:
def dependent_column(data,column):
    """
    Removes all the Next Day columns.
    Removes all the non Growth Rate Columns (GR)
    add the predictor column to list of columns.

    Parameters
    ----------
    data : dataframe

    column : string
        name of the predictor column 

    Returns
    -------
    data : dataframe
        an updated dataframe after performing all the opertaions.
    column : string
        name of the predictor column
    """
    cols = [col for col in data.columns if "next" not in col.lower() and col.lower().endswith("gr")]
    cols.append(column)
    data = data[cols]
    return (data,column)

In [ ]:
def reshape_data(x_train, x_test, y_train, y_test):
    x_train = np.array(x_train)
    x_train = np.reshape(x_train, (x_train.shape[0],1, x_train.shape[1]))
    x_test = np.array(x_test)
    x_test = np.reshape(x_test, (x_test.shape[0],1, x_test.shape[1]))
    y_test = np.array(y_test)
    y_test = np.reshape(y_test, (y_test.shape[0],1))

    return (x_train, x_test, y_train, y_test)

In [ ]:
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))

In [ ]:
def build_lstm(X_train,layers,units = 50):
    model = Sequential()
    model.add(LSTM(units=units,return_sequences=True,input_shape=(1,X_train.shape[1])))
    model.add(Dropout(0.2))
    for i in range(layers):
        model.add(LSTM(units=units,return_sequences=True))
        model.add(Dropout(0.2))
    model.add(Dense(units = 1,activation = 'relu'))
    return model

In [ ]:
def keras_model(input_dim,units = 8):
    activation='relu'
    dropout_rate=0.2
    init_mode='uniform'
    weight_constraint=0 
    optimizer='adam' 
    lr = 0.01
    momemntum=0
    
    model = Sequential()
    model.add(Dense(units = units, input_dim = input_dim, kernel_initializer=init_mode,activation=activation))
    model.add(Dropout(dropout_rate)) 
    model.add(Dense(1, kernel_initializer=init_mode, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [ ]:
def tuning(model,epochs,batch_size,X_train,Y_train):
    param_grid = dict(epochs=epochs, batch_size=batch_size)
    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1,verbose=0)
    grid_result = grid.fit(X_train,Y_train) 
    epoch = grid_result.best_params_['epochs']
    batch_size = grid_result.best_params_['batch_size']
    
    return (epoch,batch_size)

In [ ]:
def create_confusion_matrix(y_pred,y_true):
    
    cm = confusion_matrix(y_true,y_pred)
    accuracy = metrics.accuracy_score(y_true,y_pred)
    precision = metrics.precision_score(y_true,y_pred)
    recall = metrics.recall_score(y_true,y_pred)
    f1_score = metrics.f1_score(y_true,y_pred)
    return {"accuracy":accuracy,"precision":precision,"recall":recall,"f1_score":f1_score,"confusion matrix":cm}

In [ ]:
combinations = list()
def get_combinations(arr):
    n = len(arr)
    indices = [0 for i in range(n)]
    while (1):
        res = list()
        for i in range(n):
            res.append(arr[i][indices[i]])
        combinations.append(res)
        next = n - 1
        while (next >= 0 and
              (indices[next] + 1 >= len(arr[next]))):
            next-=1
        if (next < 0):
            return
        indices[next] += 1
        for i in range(next + 1, n):
            indices[i] = 0
u = [25,50,100]
b = [25,32,64]
e = [25,50,100]
t = [0.01,0.02,0.03,0.04]

arr = [u,b,e,t]
get_combinations(arr)

In [ ]:
def recurrent_neural_networks_classification(df,column = "Next Day Close Price GR",layers=3,epochs = 5,batch_size = 32,units=50,threshold = 0.01):
    defval = {"batch_size":batch_size,"epochs":epochs,units:"units"}
    df["Target"] = df[column].apply(lambda x : 1 if x >= threshold else 0)
    X = df.drop(columns=["Target",column])
    Y = df["Target"]
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,random_state = 0)
#     x_train, x_test, y_train, y_test = reshape_data(x_train, x_test, y_train, y_test)
    input_dim = x_train.shape[1]
    model = KerasClassifier(build_fn = keras_model, batch_size=batch_size, epochs=epochs,input_dim=input_dim) 
    batch_size = [25,32,64]
    epochs = [25,50,100]
    
    (epochs,batch_size) = tuning(model,epochs,batch_size,x_train,y_train)
    
    epochs = [epochs-5,epochs+5]
    batch_size = [batch_size-5, batch_size+5]

    (epochs,batch_size) = tuning(model,epochs,batch_size,x_train,y_train)
    
    
    model_lstm = build_lstm(x_train,layers,units)
    
    model_lstm.compile(optimizer='adam',loss='mse',metrics=['accuracy'])
    
    history_lstm = model.fit(x_train,y_train,epochs = epochs,batch_size = batch_size, validation_data = (x_test,y_test),shuffle = False,verbose=0)

    y_pred = model.predict(x_test)
    y_pred = np.array(y_pred)
    y_pred = np.reshape(y_pred, (y_pred.shape[0],1))

    rmse = sqrt(metrics.mean_squared_error(y_test, y_pred))
    mae = metrics.mean_absolute_error(y_test, y_pred)
    mse = metrics.mean_squared_error(y_test, y_pred)
    r2 = metrics.r2_score(y_test, y_pred)

    myres = create_confusion_matrix(y_pred,y_test)
    myres.update({"root_mean_squared_error":rmse,"mean_absolute_error":mae,"mean_squared_error":mse,"rsquared_adj":r2})
    myres.update({"threshold":threshold})
    myres.update(defval)
    myres.update({"batch_size_tune":batch_size,"epochs_tune":epochs,units:"units"})
    print("done")
    return myres

In [ ]:
layers = 3
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(filename)
        filepath = os.path.join(dirname, filename)
        df = pd.read_csv(filepath)
        column = "Next Day Close Price GR"
        df = pre_process_data(df,60)
        (df,column) = dependent_column(df,column)
        arguments = list()
        for units,batch_size,epochs,threshold in combinations:
            data = [df,column,layers,epochs,batch_size,units,threshold]
            arguments.append(data)
        threads = ThreadPool(4)
        result = threads.starmap(recurrent_neural_networks_classification,arguments[:4])
        resultdf = pd.DataFrame(result)
        resultdf.to_csv(os.path.join(os.getcwd(),str(filename[2:8])+"_class"+".csv"),index=None)
        break